In [1]:
import pandas as pd
import numpy as np

In [2]:
#modify the JSON input file name to be converted as necessary
#the below file is a sample

import json
with open('Discord Gophers-golang-Aug2020-Sep2020.json',encoding='utf8') as f:
    data = json.load(f)

In [3]:
#removing unnecessary information from the JSON

if 'guild' in data:
   del data['guild']
if 'channel' in data:
   del data['channel']
if 'dateRange' in data:
   del data['dateRange']
if 'messageCount' in data:
   del data['messageCount']

In [4]:
for key in data.keys():
    print (key)

messages


In [5]:
messages  = data['messages']

In [6]:
x = str(messages[0]).split()

In [7]:
#helper functions are as follow to extract user name, timestamp and message content

def checkstrmsg(x,start,end):
    final = []
    stringnew = ""
    msg = []
    for z in range(start+1,end):
        msg.append(x[z].strip(",\'\""))
        stringnew = ' '.join(msg)
    final.append(stringnew)
    return final

In [8]:
startlist = []
endlist = []
final = []
msg = []
finalcontent = []
for i in range(0,len(messages)):
    stringnew = ""
    final = []
    new = str(messages[i]).split()
    targetindex = new.index("'attachments':")
    new = new[:targetindex]
    
    for i in range(0,len(new)):
        if new[i] == "'content':": 
            linest = i
              
        if new[i] == "'author':" :
            lineend = i
    content = checkstrmsg(new,linest,lineend)
    finalcontent.append(content)

In [9]:
def checkname(x):
    name = []
    for i in range (0,len(x)):
        if x[i] =="'author':":
            
            name.append(x[i+4].strip(",\'\""))
    return name

In [10]:
def checkstrid(x):
    id = []
    for i in range(0,len(x)):
        if 'id' in x[i] and x[i+2] == "'type':":
            id.append(x[i+1].strip(",\'\""))
    return id
    

In [11]:
def checkstrtimestamp(x):
    tstmp = []
    for i in range(0,len(x)):
        if x[i] == "'timestamp':" and x[i+2] == "'timestampEdited':":
            tstmp.append(x[i+1].strip(",\'\""))
    return tstmp

In [12]:
id = []
names = []
ts = []

for i in range(0,len(messages)):
    new = str(messages[i]).split()
    targetindex = new.index("'attachments':")
    new = new[:targetindex]
    data_id = checkstrid(new)
    data_names = checkname(new)
    data_ts = checkstrtimestamp(new)
    id.append(data_id)
    names.append(data_names)
    ts.append(data_ts)

In [13]:
print(len(id),len(names),len(ts))

60262 60262 60262


In [14]:
data_list = list(id)
df = pd.DataFrame(data_list)
data_list2 = list(ts)

df2 = pd.DataFrame(ts)
df3 = pd.DataFrame(finalcontent)
df4 = pd.DataFrame(names)

In [15]:
df = df.iloc[: , :1]
df2 = df2.iloc[: , :1] 
df3 = df3.iloc[: , :1] 
df4 = df4.iloc[: , :1]

In [16]:
dat2 = pd.concat([df2,df4,df3], axis=1)

In [17]:
dat2.columns = [ 'ts' , 'user' , 'text']

In [18]:
dat2['text'] = dat2['text'].str.replace('*','')
dat2['text'] = dat2['text'].replace(r'\\n','', regex=True)
dat2['text'] = dat2['text'].replace(r'-','', regex=True)
dat2["ts"] = dat2["ts"].apply(lambda x: x.replace("+00:00", ""))

In [19]:
dat2['ts'] = pd.to_datetime(dat2['ts']).dt.strftime("%Y-%m-%dT%H:%M:%S.%f")

In [20]:
filterinfDataframe = dat2[(dat2['ts'] >= "2020-08-01") ] #add respective start date range

In [21]:
filterinfDataframe = filterinfDataframe[(filterinfDataframe['ts']< "2020-10-01") ] #add respective end date range

In [22]:
filterinfDataframe.count()

ts      60079
user    60079
text    60079
dtype: int64

In [23]:
#changing the dataframe into dictionary

dat2_2 = filterinfDataframe.to_dict('records')

In [24]:
#making the dictionary into ordered dictionary to preserve the xml tag order

from collections import OrderedDict
def to_od(d):
   if not isinstance(d, (dict, list)):
      return d
   if isinstance(d, list):
       return list(map(to_od, d))
   return OrderedDict({a:to_od(b) for a, b in d.items()})

dat3 = to_od(dat2_2)

In [25]:
# modify the respective temp file name as necessary

from dict2xml import dict2xml
xml = dict2xml(dat3, wrap ='message', indent ="  ")
f = open('golang_Aug2020-Sep2020temp.xml','w', encoding='utf8')
print(xml,file=f)

In [26]:
# modify the respective temp file name as necessary

with open('golang_Aug2020-Sep2020temp.xml', 'r',encoding='utf8') as f:
    data = f.read()

with open('root.xml', 'w',encoding='utf8') as f:
    f.write("<discord>\n" + data + "</discord>")
    f.close()

In [27]:
#Script to add the xml file info at the start

import xml.etree.ElementTree as ET
with open('root.xml', 'r', encoding='utf-8') as f:
    content = f.read()
root = ET.fromstring(content)

for message in root.findall('message'):
    last = root[-1]
    first = root[0]
    tsfirst = first.find('ts').text
    tslast = last.find('ts').text

In [28]:
# modify the respective temp file name as necessary

with open('golang_Aug2020-Sep2020temp.xml', 'r',encoding='utf8') as f:
    data = f.read()

with open('golang_Aug2020-Sep2020temp.xml', 'w',encoding='utf8') as f:
    f.write("<team_domain>Discord Gophers</team_domain>\n" + "<channel_name>golang</channel_name>\n" +
            "<start_date>"+tsfirst+"</start_date>\n" + "<end_date>"+tslast+"</end_date>\n" 
            + data)
    f.close()

In [29]:
# modify the respective temp file name as necessary

with open('golang_Aug2020-Sep2020temp.xml', 'r',encoding='utf8') as f:
    data = f.read()

with open('golang_Aug2020-Sep2020temp.xml', 'w',encoding='utf8') as f:
    f.write("<discord>\n" + data + "</discord>")
    f.close()

In [30]:
#script to anonymize user name for privacy purposes
# modify the respective temp file and final xml file names as necessary

import xml.etree.ElementTree as ET
import sys
import re
from random import shuffle
from datetime import datetime


def replaceMentions(text, aliasDict):
    text = str(text)
    mentionPattern = '<@U\w+>'
    while re.search(mentionPattern,text):
        indexStart = re.search(mentionPattern,text).span()[0]
        indexEnd = re.search(mentionPattern,text).span()[1]
        mentionName = text[indexStart+2:indexEnd-1]
        try:
            mentionAlias = aliasDict[mentionName]
        except KeyError:
            aliasDict[mentionName] = names.pop()
            mentionAlias = aliasDict[mentionName]
        text = text[:indexStart] + "<@" + mentionAlias + ">"+ text[indexEnd:]
    return text

names_file = 'names.txt'
original_chat = 'golang_Aug2020-Sep2020temp.xml'
output_fname = 'golang_Aug2020-Sep2020.xml'


names = []
with open(names_file, 'r') as f:
    names = [x.rstrip().title() for x in f.readlines()]
shuffle(names)
aliasDict = {}

with open(original_chat, 'r', encoding='utf-8') as f:
    content = f.read()
root = ET.fromstring(content)

# first 4 elements are the team, channel, start date/time, and end date/time
info = root[:4]
nodes = root[4:]

# format input
TSfmt = '%Y-%m-%dT%H:%M:%S.%f'	# 24-hour time format
starttime = datetime.strptime(nodes[0][0].text, TSfmt)
outputStr = ""
print("Total messages: " + str(len(nodes)) + "\n", file=sys.stderr)
for node in nodes:
    timestamp = node[0].text
    tdelta = datetime.strptime(timestamp, TSfmt) - starttime
    secondsFromStart = round(tdelta.total_seconds())

    user = node[1].text
    text = node[-1].text

    alias = user
    try:
        alias = aliasDict[user]
    except KeyError:
        aliasDict[user] = names.pop()
        alias = aliasDict[user]

    node[1].text = alias
    node[-1].text = replaceMentions(text,aliasDict)


with open(output_fname, 'w',encoding="utf-8") as f:
    f.write(ET.tostring(root,encoding="utf-8",method="xml").decode('utf-8'))

Total messages: 60079

